### Fiona Fan
Problem 1

In [3]:
import numpy as np
from scipy import linalg as la
from numpy.linalg import matrix_power

In [4]:
def f(A,k,A_D):
    if np.allclose(np.dot(A,A_D),np.dot(A_D, A))==True:
        if np.allclose(np.dot(matrix_power(A,k+1),A_D), matrix_power(A,k))==True:
            if np.allclose(np.dot(A_D,A,A_D),A_D)==True:
                return True
            else:return False
        else: return False
    else: return False
    

In [5]:
A=np.array(([1,3,0,0],[0,1,3,0],[0,0,1,3],[0,0,0,0]))
A.reshape(4,4)
A_D=np.array(([1,-3,9,81],[0,1,-3,-18],[0,0,1,3],[0,0,0,0]))
A_D.reshape(4,4)
f(A,1,A_D)

True

In [6]:
B=np.array(([1,1,3],[5,2,6],[-2,-1,-3]))
B.reshape(3,3)
B_D=np.array(([0,0,0],[0,0,0],[0,0,0]))
B_D.reshape(3,3)
f(B,3,B_D)

True

Problem 2

In [7]:
from numpy.linalg import inv
def p2(A,tol):
    n=len(A)
    f = lambda x: abs(x) > tol
    Q1,S,k1 = la.schur(A, sort=f)
    f = lambda x: abs(x) <= tol
    Q2,T,k2 = la.schur(A, sort=f)
    U=np.concatenate((S[:,:k1], T[:,:n-k1]), axis=1)
    U_1=inv(U)
    V=U_1@A@U
    Z=np.zeros((n,n))
    if k1!=0:
        M_1=inv(V[:k1,:k1])
        Z[:k1,:k1]=M_1
    return U@Z@U_1
    

In [8]:
p2(A,0.01)

array([[  1.,  -3.,   9.,  81.],
       [  0.,   1.,  -3., -18.],
       [  0.,   0.,   1.,   3.],
       [  0.,   0.,   0.,   0.]])

In [9]:
f(A,1,p2(A,0.01))

True

Problem 3

In [10]:
def p3(A):
    n =len(A)
    degree = np.sum(A, axis = 0)
    D = np.diag(degree)
    L = D - A
    I = np.identity(n)
    R = np.zeros((n,n))
    for j in range(n):
        L1 = L.copy()
        L1[:, j] = I[:, j]
        L1_D = p2(L1, .0000001)
        R[:, j] = L1_D.diagonal()
        R[j, j] = 0
    return R


In [11]:
A=np.array([[0, 1, 0, 0], [1,  0,  1,  0], [0,  1,  0,  1], [0,  0,  1,  0]])

p3(A)

array([[0., 1., 2., 3.],
       [1., 0., 1., 2.],
       [2., 1., 0., 1.],
       [3., 2., 1., 0.]])

Problem 4&5

In [12]:
import pandas as pd
class LinkPredictor:
    def __init__(self, name):
        data= pd.read_csv(name, header=None)
        self.list = np.unique(np.concatenate((data[0], data[1])))
        n = len(self.list)
        adj = np.zeros((n, n))
        for i in data.index:
            friend1, friend2 = data[0][i], data[1][i]
            friend1loc = np.where(friend1 == self.list)
            friend2loc = np.where(friend2 == self.list)
            adj[friend1loc, friend2loc] = 1
            adj[friend2loc, friend1loc] = 1
        self.adjacency_matrix = adj
        self.resistance_matrix = p3(self.adjacency_matrix)
    def predict_link(self,node):
        #zero out all entries that represent connected nodes
        new_matrix=self.resistance_matrix*(1-self.adjacency_matrix)
        if node=='None':
            minval = np.min(new_matrix[np.nonzero(new_matrix)])
            loc= np.where(new_matrix==minval)
            return self.list[loc[0]], self.list[loc[1]]
        elif not node in self.list:
            raise ValueError("invalid input")
        else :
            loc = np.where(self.list == node)
            col = new_matrix[:,loc]
            minval = np.min(col[col > 0])
            location = np.where(col==minval)
            return self.list[location[0]]
    def add_link(self,node1,node2):
        if (not node1 in self.list) or (not node2 in self.list):
            raise ValueError("invalid inputs")
        else:
            loc1 = np.where(self.list==node1)
            loc2 = np.where(self.list==node2)
            self.adjacency_matrix[loc1, loc2] = 1
            self.adjacency_matrix[loc2, loc1] = 1

In [13]:
a=LinkPredictor("social_network.csv")
a.predict_link('Melanie')

array(['Carol'], dtype=object)

In [14]:
a.predict_link('Alan')

array(['Sonia'], dtype=object)

In [15]:
a.predict_link('None')

(array(['Emily', 'Oliver'], dtype=object),
 array(['Oliver', 'Emily'], dtype=object))

In [16]:
a.add_link('Emily','Alan')